# Processing III: motion tracking and balance

In this notebook, we work further with motion tracking data extracted in XXX. Additionally, we process the balance board stream that we extracted in XX. 

In [ ]:
# packages
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy


curfolder = os.getcwd()
print(curfolder)

# files to work with
MTfolder = 'C:\\Users\\kadava\\Documents\\Github\\FLESH_3Dtracking_new\\projectdata\\' ## FLAGGED CHANGE
ACfolder = 'E:\\charade_experiment_WORKSPACE\\xdf_procedure\\data\\Data_processed\\Data_trials\\Audio_48\\' ## FLAGGED CHANGE
BBfolder = 'E:\\charade_experiment_WORKSPACE\\xdf_procedure\\data\\Data_processed\\Data_trials\\'

# folders to save the processed data
MTfolder_processed = curfolder + '\\TS_motiontracking\\'
ACfolder_processed = curfolder + '\\TS_acoustics\\'
TSmerged = curfolder + '\\TS_merged\\'

e:\FLESH_ContinuousBodilyEffort\01_TS_processing


# Motion tracking 

FLAGGED. Note that y and z dimensions are swapped (such that z is vertical)

In the previous notebook, we have ran pose estimation on the trial videos (OpenPose), and triangulated the coordinates to get 3D coordinates for each trial (pose2sim). Furthermore, we have performed inverse kinematics and dynamics to extract joint angles and moments.

In this section, we will clean the data, and extract further information (such as speed, acceleration, etc.). 

## Motion tracking - kinematics

In [9]:
MTtotrack = glob.glob(MTfolder + '*/P*/*', recursive=True)

# get rid of all the folders that are not the ones we want to track, like .sto files
MTtotrack = [x for x in MTtotrack if 'sto' not in x]
MTtotrack = [x for x in MTtotrack if 'txt' not in x]
MTtotrack = [x for x in MTtotrack if 'xml' not in x]
MTtotrack = [x for x in MTtotrack if 'opensim' not in x]
MTtotrack = [x for x in MTtotrack if 'Results' not in x]
MTtotrack = [x for x in MTtotrack if 'toml' not in x]

print(MTtotrack)

MTfiles_all = []

for folder in MTtotrack:
    print('working on:' + folder)
    # last element is trialid
    trialid = folder.split('\\')[-1]
    
    # get all csv files in the folder
    csvfiles = glob.glob(folder + '\\**\\*.csv', recursive=True)
    # keep only the ones that have butterworth in the name
    csvfiles = [x for x in csvfiles if 'butterworth' in x]
    butterfile = csvfiles[0]
    # append to list with trialid
    MTfiles_all.append([trialid, butterfile])

[]


The data can still be noisy, as the OpenPose in-built filter is not particularly strong. We don't want to smooth all the keypoints with the same strength, as some keypoints are more prone to noise than others. We can therefore use the function `check_smooth_strength` to check the effect of different smoothing strengths on the data. 

In [2]:
# function to check different smoothing windows and orders
def check_smooth_strength(df, keycols, windows, orders):

    # prepare new df
    df_smooth = pd.DataFrame()

    for col in keycols:
        for win in windows:
            for ord in orders:
                df_smooth[col + '_savgol' + str(win) + '_' + str(ord)] = scipy.signal.savgol_filter(df[col], win, ord)

    # make R_Hand_x from df_sample a list
    keycol_x = df[keycols[0]].tolist()
    keycol_y = df[keycols[1]].tolist()
    keycol_z = df[keycols[2]].tolist()

    # load these values into df_smooth as a new column
    df_smooth[keycols[0]] = keycol_x
    df_smooth[keycols[1]] = keycol_y
    df_smooth[keycols[2]] = keycol_z

    # plot all dimensions in one plot
    colstoplot = keycols

    for col in colstoplot:
        plt.plot(df_smooth[col], label=col, alpha=0.65)
    plt.legend()
    # make only timewindow 50 to 250
    #plt.xlim(50, 222)
    plt.show()

### 15, 1 seems quite solid

# but for lower body let's try 20,1

In [3]:
print(MTfiles_all[20][1])
sample = pd.read_csv(MTfiles_all[20][1], sep=',')

windows = [50,70,100] # list possible window
orders = [1,3] # list possible orders

# col of interest
samplecol = ['LKnee_x', 'LKnee_y', 'LKnee_z']

check_smooth_strength(sample, samplecol, windows, orders)

NameError: name 'MTfiles_all' is not defined

In [ ]:
# function to get euclidian sum of associated keypoints

def aggregate_keypoints(df, measurement, finalcolname):

    # group keypoints that belong together
    lowerbodycols = ['RHip', 'LHip']
    legcols = ['RKnee', 'RAnkle', 'LAnkle', 'LKnee', 'RHeel', 'LHeel']
    headcols = ['Head', 'Neck', 'Nose']
    armcols = ['RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'RIndex', 'LIndex']

    groups = [lowerbodycols, legcols, headcols, armcols]

    # make subdf only with speed
    subdf = df[[x for x in df.columns if measurement in x]]

    # loop through each joint group
    for group in groups:
        # get cols
        cols = [x for x in subdf.columns if any(y in x for y in group)]
        subdf_temp = subdf[cols]

        for index, row in subdf_temp.iterrows():
            # get all values of that row
            values = row.values
            # calculate euclidian sum
            euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize
            # get a name for new col
            if group == lowerbodycols:
                colname = 'lowerbody'
            elif group == legcols:
                colname = 'leg'
            elif group == headcols:
                colname = 'head'
            elif group == armcols:
                colname = 'arm'

            df.loc[index, colname + finalcolname] = euclidian_sum

    return df


# get kinematic derivatives
def get_derivatives(df, sr, upperbodycols, lowerbodycols):

    mtcols = df.columns

    # prepare cols for speed
    cols = [x.split('_')[0] for x in mtcols]
    colsforspeed = list(set(cols))

    # for each unique colname (cols), calculate speed 
    for col in colsforspeed:
        # get x and y columns
        x = df[col + '_x']
        y = df[col + '_y']
        z = df[col + '_z'] # note that y and z are flipped
        # calculate speed
        df[col + '_speed'] = np.insert(np.sqrt(np.diff(x)**2 + np.diff(y)**2 + np.diff(z)**2), 0, 0)
        # multiply the values by sr, because now we have values in m/(s/sr)
        df[col + '_speed'] = df[col + '_speed']*sr
        
        # smooth
        if any(x in col for x in upperbodycols):
            df[col + '_speed'] = scipy.signal.savgol_filter(df[col + '_speed'], 15, 1)
        elif any(x in col for x in lowerbodycols):
            df[col + '_speed'] = scipy.signal.savgol_filter(df[col + '_speed'], 20, 1)
        else:
            df[col + '_speed'] = scipy.signal.savgol_filter(df[col + '_speed'], 15, 1)

        # if the col contains wrist, we will alco calculate the vertical velocity (z dimension)
        if 'Wrist' in col:
            # calculate speed
            df[col + '_vert_vel'] = np.insert(np.diff(z), 0, 0)
            # multiply the values by sr, because now we have values in m/(s/sr)
            df[col + '_vert_vel'] = df[col + '_vert_vel']*sr
            # smooth with savgol 
            df[col + '_vert_vel'] = scipy.signal.savgol_filter(df[col + '_vert_vel'], 15, 1)

        # derive acceleration	
        df[col + '_acc'] = np.insert(np.diff(df[col + '_speed']), 0, 0)
        # smooth
        df[col + '_acc'] = scipy.signal.savgol_filter(df[col + '_acc'], 15, 1)

        # derive jerk
        df[col + '_jerk'] = np.insert(np.diff(df[col + '_acc']), 0, 0)
        # smooth
        df[col + '_jerk'] = scipy.signal.savgol_filter(df[col + '_jerk'], 15, 1)

    return df


In [ ]:
for folder in MTtotrack:
    print('working on:' + folder)
    # last element is trialid
    trialid = folder.split('\\')[-1]
    
    # get all csv files in the folder
    csvfiles = glob.glob(folder + '/**/*.csv', recursive=True)
    # keep only the ones that have butterworth in the name
    csvfiles = [x for x in csvfiles if 'butterworth' in x]
    butterfile = csvfiles[0]

    # load it
    mt = pd.read_csv(butterfile)

    # the mt is missing 0 ms timepoint, so we need to create a row that copies the first row of mt and time = 0
    padrow = mt.iloc[0].copy()
    padrow['Time'] = 0

    # concatenate it to the beginning of mt 
    mt = pd.concat([pd.DataFrame(padrow).T, mt], ignore_index=True)

    # keep only cols of interest
    colstokeep = ["Time", "RHip", "RKnee", "RAnkle", "RHeel", "LHip", "LKnee", "LAnkle", "LHeel", "Neck", "Head", "Nose", "RShoulder", "RElbow", "RWrist", "RIndex", "LShoulder", "LElbow", "LWrist",
    "LIndex",
]
    mt = mt[[col for col in mt.columns if any(x in col for x in colstokeep)]]

    # show columns
    mtcols = mt.columns

    # smooth all columns except time with savgol
    colstosmooth = mtcols[:-1]

    mt_smooth = pd.DataFrame()

    # upper body cols
    upperbodycols = ['Head', 'Neck', 'RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'Nose', 'RIndex', 'LIndex']
    # lower body cols
    lowerbodycols = ['RHip', 'RKnee', 'RAnkle', 'RHeel' 'LHip', 'LKnee', 'LAnkle', 'LHeel']

    for col in colstosmooth:
        # if the col + x/y/z is in upperbodycols, smooth with 15,1
        if any(x in col for x in upperbodycols):
            mt_smooth[col] = scipy.signal.savgol_filter(mt[col], 15, 1)
        # as the lowerbody keypoints are not moving that much, they are much more prone to noise (e.g., from the measurement error of OpenPose, therefore we will smooth them with a little higher window)
        elif any(x in col for x in lowerbodycols):
            mt_smooth[col] = scipy.signal.savgol_filter(mt[col], 20, 1)
        else:
            mt_smooth[col] = scipy.signal.savgol_filter(mt[col], 15, 1)

        # and put them all to cms
        mt_smooth[col] = mt_smooth[col]*100

    # add back time column
    mt_smooth['Time'] = mt['Time']

    # get sampling rate
    sr = 1/np.mean(np.diff(mt['Time']))

    mt_smooth = get_derivatives(mt_smooth, sr, upperbodycols, lowerbodycols)
    # prepare cols for speed
    # cols = [x.split('_')[0] for x in mtcols]
    # colsforspeed = list(set(cols))

    # # for each unique colname (cols), calculate speed 
    # for col in colsforspeed:
    #     # get x and y columns
    #     x = mt_smooth[col + '_x']
    #     y = mt_smooth[col + '_y']
    #     z = mt_smooth[col + '_z'] # note that y and z are flipped
    #     # calculate speed
    #     mt_smooth[col + '_speed'] = np.insert(np.sqrt(np.diff(x)**2 + np.diff(y)**2 + np.diff(z)**2), 0, 0)
    #     # multiply the values by sr, because now we have values in m/(s/sr)
    #     mt_smooth[col + '_speed'] = mt_smooth[col + '_speed']*sr
        
    #     # smooth
    #     if any(x in col for x in upperbodycols):
    #         mt_smooth[col + '_speed'] = scipy.signal.savgol_filter(mt_smooth[col + '_speed'], 15, 1)
    #     elif any(x in col for x in lowerbodycols):
    #         mt_smooth[col + '_speed'] = scipy.signal.savgol_filter(mt_smooth[col + '_speed'], 20, 1)
    #     else:
    #         mt_smooth[col + '_speed'] = scipy.signal.savgol_filter(mt_smooth[col + '_speed'], 15, 1)

    #     # if the col contains wrist, we will alco calculate the vertical velocity (z dimension)
    #     if 'Wrist' in col:
    #         # calculate speed
    #         mt_smooth[col + '_vert_vel'] = np.insert(np.diff(z), 0, 0)
    #         # multiply the values by sr, because now we have values in m/(s/sr)
    #         mt_smooth[col + '_vert_vel'] = mt_smooth[col + '_vert_vel']*sr
    #         # smooth with savgol 
    #         mt_smooth[col + '_vert_vel'] = scipy.signal.savgol_filter(mt_smooth[col + '_vert_vel'], 15, 1)

    #     # derive acceleration	
    #     mt_smooth[col + '_acc'] = np.insert(np.diff(mt_smooth[col + '_speed']), 0, 0)
    #     # smooth
    #     mt_smooth[col + '_acc'] = scipy.signal.savgol_filter(mt_smooth[col + '_acc'], 15, 1)

    #     # derive jerk
    #     mt_smooth[col + '_jerk'] = np.insert(np.diff(mt_smooth[col + '_acc']), 0, 0)
    #     # smooth
    #     mt_smooth[col + '_jerk'] = scipy.signal.savgol_filter(mt_smooth[col + '_jerk'], 15, 1)

    # getting aggreagated sums for groups of cols
    mt_smooth = aggregate_keypoints(mt_smooth, 'speed', 'speedKin_sum')
    mt_smooth = aggregate_keypoints(mt_smooth, 'acc', 'accKin_sum')
    mt_smooth = aggregate_keypoints(mt_smooth, 'jerk', 'jerkKin_sum')

    # lowerbodycols = ['RHip', 'LHip']
    # legcols = ['RKnee', 'RAnkle', 'LAnkle', 'LKnee', 'RHeel', 'LHeel']
    # headcols = ['Head', 'Neck', 'Nose']
    # armcols = ['RShoulder', 'RElbow', 'RWrist', 'LShoulder', 'LElbow', 'LWrist', 'RIndex', 'LIndex']

    # groups = [lowerbodycols, legcols, headcols, armcols]

    # # make subdf only with speed
    # subdf = mt_smooth[[x for x in mt_smooth.columns if 'speed' in x]]

    # # loop through each joint group
    # for group in groups:
    #     # get cols
    #     cols = [x for x in subdf.columns if any(y in x for y in group)]
    #     subdf_temp = subdf[cols]

    #     for index, row in subdf_temp.iterrows():
    #         # get all values of that row
    #         values = row.values
    #         # calculate euclidian sum
    #         euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize
    #         # get a name for new col
    #         if group == lowerbodycols:
    #             colname = 'lowerbody'
    #         elif group == legcols:
    #             colname = 'leg'
    #         elif group == headcols:
    #             colname = 'head'
    #         elif group == armcols:
    #             colname = 'arm'

    #         mt_smooth.loc[index, colname + '_speedKin_sum'] = euclidian_sum

    # # now for acc
    # subdf = mt_smooth[[x for x in mt_smooth.columns if 'acc' in x]]

    # # loop through each joint group
    # for group in groups:

    #     # get cols
    #     cols = [x for x in subdf.columns if any(y in x for y in group)]

    #     subdf_temp = subdf[cols]

    #     for index, row in subdf_temp.iterrows():
    #         # get all values of that row
    #         values = row.values

    #         # calculate euclidian sum
    #         euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize

    #         # get a name for new col
    #         if group == lowerbodycols:
    #             colname = 'lowerbody'
    #         elif group == legcols:
    #             colname = 'leg'
    #         elif group == headcols:
    #             colname = 'head'
    #         elif group == armcols:
    #             colname = 'arm'


    #         mt_smooth.loc[index, colname + '_accKin_sum'] = euclidian_sum

    # # now for jerk
    # subdf = mt_smooth[[x for x in mt_smooth.columns if 'jerk' in x]]

    # # loop through each joint group
    # for group in groups:

    #     # get cols
    #     cols = [x for x in subdf.columns if any(y in x for y in group)]

    #     subdf_temp = subdf[cols]

    #     for index, row in subdf_temp.iterrows():
    #         # get all values of that row
    #         values = row.values

    #         # calculate euclidian sum
    #         euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize

    #         # get a name for new col
    #         if group == lowerbodycols:
    #             colname = 'lowerbody'
    #         elif group == legcols:
    #             colname = 'leg'
    #         elif group == headcols:
    #             colname = 'head'
    #         elif group == armcols:
    #             colname = 'arm'


    #         mt_smooth.loc[index, colname + '_jerkKin_sum'] = euclidian_sum

    # add trialid
    mt_smooth['TrialID'] = trialid
    # convert time to ms
    mt_smooth['Time'] = mt_smooth['Time']*1000
    # write to csv
    mt_smooth.to_csv(MTfolder_processed + '/mt_' + trialid + '.csv', index=False)


Let's check one file to see how the data looks like

In [ ]:
# load one in to check

MTfiles = glob.glob(MTfolder_processed + '/*.csv')
print(MTfiles)

sample = pd.read_csv(MTfiles[0])
print(sample)

# FLAGGED - plot here coordinates-speed-acc-jerk

## Motion tracking - inverse kinematics

Using OpenSim, we have extracted joint angles in the previous notebook. Now again, we clean the data and extract further information before saving it into csv file per trial

In [ ]:
# get all mot files in the folder
mot_files = glob.glob(MTfolder + '*/P*/*/*.mot', recursive=True)

for mot in mot_files:
    print('working on ' + mot)
    # get trialid
    trialid = mot.split('\\')[-1].split('.')[0]

    # get rid of the first element before _
    trialid = '_'.join(trialid.split('_')[1:])

    # load it
    mot_df = pd.read_csv(mot, sep='\t', skiprows=10)
    
    # pad 0 ms row
    padrow = mot_df.iloc[0].copy()
    padrow['time'] = 0

    # concatenate it to the beginning of mot_df
    mot_df = pd.concat([pd.DataFrame(padrow).T, mot_df], ignore_index=True)
    
    # get the sr
    sr = 1/np.mean(np.diff(mot_df['time']))

    # smooth all columns except the firts time (time) and last (trialid)
    colstosmooth = [x for x in mot_df.columns if 'time' not in x]

    # smooth
    for col in colstosmooth:
        mot_df[col] = scipy.signal.savgol_filter(mot_df[col], 15, 3)
        # convert to radians
        mot_df[col] = np.deg2rad(mot_df[col])

    # convert time to ms
    mot_df['time'] = mot_df['time']*1000

    # keep only columns you might use
    keypoints = ['wrist', 'pro', 'elbow', 'arm', 'neck', 'ankle', 'knee', 'hip', 'pelvis']
    coi = [x for x in mot_df.columns if any(y in x for y in keypoints) or 'time' in x or 'TrialID' in x]
    mot_df2 = mot_df[coi]

    # get derivatives
    mot_df2 = get_derivatives(mot_df2, sr, [], [])
    
    # calculate angular speed
    speedcols = [x for x in mot_df2.columns if 'time' not in x]
    speedcols = [x for x in speedcols if 'TrialID' not in x]
    print(speedcols)

    for col in speedcols:
        mot_df2[col + '_speed'] = np.insert(np.diff(mot_df2[col]), 0, 0)
        mot_df2[col + '_speed'] = mot_df2[col + '_speed']*sr
        mot_df2[col + '_speed'] = scipy.signal.savgol_filter(mot_df2[col + '_speed'], 15, 1)

    # calculate angular acceleration
    acccols = [x for x in mot_df2.columns if 'speed' in x]

    for col in acccols:
        # replace speed with ''
        col_name = col.replace('_speed', '')
        mot_df2[col_name + '_acc'] = np.insert(np.diff(mot_df2[col]), 0, 0)
        mot_df2[col_name + '_acc'] = scipy.signal.savgol_filter(mot_df2[col_name + '_acc'], 15, 1)

    # calculate jerk
    jerkcols = [x for x in mot_df2.columns if 'acc' in x]

    for col in jerkcols:
        # replace acc with ''
        col_name = col.replace('_acc', '')
        mot_df2[col_name + '_jerk'] = np.insert(np.diff(mot_df2[col]), 0, 0)
        mot_df2[col_name + '_jerk'] = scipy.signal.savgol_filter(mot_df2[col_name + '_jerk'], 15, 1)


    # add time and trialid
    mot_df2['time'] = mot_df['time']
    mot_df2['TrialID'] = trialid

    # let's get aggregated euclidian sum for each joint group
    # upper body
    pelviscols = ['pelvis']
    spinecols = ['L5_S1', 'L4_L5', 'L3_L4', 'L2_L3', 'L1_L2', 'L1_T12']
    lowerbodycols = ['pelvis', 'hip']
    legcols = ['knee', 'ankle', 'subtalar']
    headcols = ['neck']
    armcols = ['arm', 'elbow', 'wrist', 'pro_sup']

    groups = [lowerbodycols, legcols, headcols, armcols, pelviscols, spinecols]

    # make subdf only with moments
    subdf = mot_df2[[x for x in mot_df2.columns if 'speed' in x]]

    # loop through each joint group
    for group in groups:

        # get cols
        cols = [x for x in subdf.columns if any(y in x for y in group)]

        subdf_temp = subdf[cols]

        for index, row in subdf_temp.iterrows():
            # get all values of that row
            values = row.values

            # calculate euclidian sum
            euclidian_sum = np.sqrt(np.sum(np.square(values))) 

            # get a name for new col
            if group == lowerbodycols:
                colname = 'lowerbody'
            elif group == legcols:
                colname = 'leg'
            elif group == headcols:
                colname = 'head'
            elif group == armcols:
                colname = 'arm'
            elif group == pelviscols:
                colname = 'pelvis'
            elif group == spinecols:
                colname = 'spine'


            mot_df2.loc[index, colname + '_angSpeed_sum'] = euclidian_sum

    # and the same for acceleration
    # make subdf only with acc
    subdf = mot_df2[[x for x in mot_df2.columns if 'acc' in x]]

    # loop through each joint group
    for group in groups:

        # get cols
        cols = [x for x in subdf.columns if any(y in x for y in group)]

        subdf_temp = subdf[cols]

        for index, row in subdf_temp.iterrows():
            # get all values of that row
            values = row.values

            # calculate euclidian sum
            euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize

            # get a name for new col
            if group == lowerbodycols:
                colname = 'lowerbody'
            elif group == legcols:
                colname = 'leg'
            elif group == headcols:
                colname = 'head'
            elif group == armcols:
                colname = 'arm'
            elif group == pelviscols:
                colname = 'pelvis'
            elif group == spinecols:
                colname = 'spine'


            mot_df2.loc[index, colname + '_angAcc_sum'] = euclidian_sum

    # and the same for jerk
    # make subdf only with jerk
    subdf = mot_df2[[x for x in mot_df2.columns if 'jerk' in x]]

    # loop through each joint group
    for group in groups:

        # get cols
        cols = [x for x in subdf.columns if any(y in x for y in group)]

        subdf_temp = subdf[cols]

        for index, row in subdf_temp.iterrows():
            # get all values of that row
            values = row.values

            # calculate euclidian sum
            euclidian_sum = np.sqrt(np.sum(np.square(values))) ## FLAGGED: possibly normalize

            # get a name for new col
            if group == lowerbodycols:
                colname = 'lowerbody'
            elif group == legcols:
                colname = 'leg'
            elif group == headcols:
                colname = 'head'
            elif group == armcols:
                colname = 'arm'
            elif group == pelviscols:
                colname = 'pelvis'
            elif group == spinecols:
                colname = 'spine'

            mot_df2.loc[index, colname + '_angJerk_sum'] = euclidian_sum

    # write to csv
    mot_df2.to_csv(MTfolder_processed + '/ik_' + trialid + '.csv', index=False)


## Motion tracking - inverse dynamics

In [ ]:
# in MTfolders, find all sto files
sto_files = glob.glob(MTfolder + '*/P*/*/*.sto', recursive=True)
sto_files = [x for x in sto_files if 'ID' in x]

for sto in sto_files:
    print('working on ' + sto)

    # load it
    id_df = pd.read_csv(sto, sep='\t', skiprows=6)

    # from the filename, get the trialid
    trialid = sto.split('\\')[-1].split('.')[0]
    trialid = '_'.join(trialid.split('_')[:-1])
    trialid = '_'.join(trialid.split('_')[1:])

    # pad 0 ms row
    padrow = id_df.iloc[0].copy()
    padrow['time'] = 0

    # concatenate it to the beginning of id_df
    id_df = pd.concat([pd.DataFrame(padrow).T, id_df], ignore_index=True)

    # add trialid
    id_df['TrialID'] = trialid

    # smooth all columns except the firts time (time) and last (trialid)
    colstosmooth = [x for x in id_df.columns if 'time' not in x]
    colstosmooth = [x for x in colstosmooth if 'TrialID' not in x]

    # smooth
    for col in colstosmooth:
        id_df[col] = scipy.signal.savgol_filter(id_df[col], 15, 3)
    
    # convert time to ms
    id_df['time'] = id_df['time']*1000

    # let's get aggregated euclidian sum for each joint group
    # upper body
    pelviscols = ['pelvis']
    spinecols = ['L5_S1', 'L4_L5', 'L3_L4', 'L2_L3', 'L1_L2', 'L1_T12']
    lowerbodycols = ['pelvis', 'hip']
    legcols = ['knee', 'ankle', 'subtalar']
    headcols = ['neck']
    armcols = ['arm', 'elbow', 'wrist', 'pro_sup']

    groups = [lowerbodycols, legcols, headcols, armcols, pelviscols, spinecols]

    # make subdf only with moments
    subdf = id_df[[x for x in id_df.columns if 'moment' in x]]

    # loop through each joint group
    for group in groups:

        # get cols
        cols = [x for x in subdf.columns if any(y in x for y in group)]

        subdf_temp = subdf[cols]

        for index, row in subdf_temp.iterrows():
            # get all values of that row
            values = row.values

            # calculate euclidian sum
            euclidian_sum = np.sqrt(np.sum(np.square(values))) 

            # get a name for new col
            if group == lowerbodycols:
                colname = 'lowerbody'
            elif group == legcols:
                colname = 'leg'
            elif group == headcols:
                colname = 'head'
            elif group == armcols:
                colname = 'arm'
            elif group == pelviscols:
                colname = 'pelvis'
            elif group == spinecols:
                colname = 'spine'

            id_df.loc[index, colname + '_torque_sum'] = euclidian_sum

    # for each torque_sum, we will also calculate the change in torque_sum
    torquestodiff = [x for x in id_df.columns if 'torque_sum' in x]

    for col in torquestodiff:
        id_df[col + '_change'] = np.insert(np.diff(id_df[col]), 0, 0)
        # absolutize
        id_df[col + '_change'] = np.abs(id_df[col + '_change'])
        # smooth
        id_df[col + '_change'] = scipy.signal.savgol_filter(id_df[col + '_change'], 20, 4)
    

    # write to csv
    id_df.to_csv(MTfolder_processed + '/id_' + trialid + '.csv', index=False)

### ID&MT check


In [ ]:
import matplotlib.pyplot as plt

# load in one id and mt file with the same trialid

idfiles = glob.glob(MTfolder_processed + '/id*.csv')
mtfiles = glob.glob(MTfolder_processed + '/mt*.csv')

id = pd.read_csv(idfiles[0])
mt = pd.read_csv(mtfiles[0])

# plot
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].plot(mt['Time'], mt['LWrist_speed'], label='LWrist_speed')
# add LElbow_speed
ax[0].plot(mt['Time'], mt['LElbow_speed'], label='LElbow_speed')
ax[0].set_title('LWrist_speed')
ax[0].set_ylabel('speed (cm/s)')
ax[0].set_xlabel('time (ms)')
ax[0].legend()

# elbow flexion
ax[1].plot(id['time'], id['elbow_flex_r_moment'], label='elbow_flex_r_moment')
ax[1].plot(id['time'], id['elbow_flex_l_moment'], label='elbow_flex_l_moment')
ax[1].set_title('wrist_flex_l_moment')
ax[1].set_ylabel('moment (Nm)')
ax[1].set_xlabel('time (ms)')
ax[1].legend()

plt.show()

# Balance Board


We do XXX

In [ ]:
# in BBfolder, find all files containing BalanceBoard

BB_files = glob.glob(BBfolder + '*BalanceBoard*.csv', recursive=True)
print(BB_files)

for bb in BB_files:
    print('working on ' + bb)
    # get trialid
    trialid = bb.split('\\')[-1].split('.')[0]
    # get the first, second, fourth, nineth elements
    trialid = '_'.join(trialid.split('_')[:2] + trialid.split('_')[3:4] + trialid.split('_')[8:9])

    # because we are going to merge on bb, we will store also more information
    fileinfo = bb.split('\\')[-1].split('.')[0]

    # if second element is 1, we will store last three elements
    if fileinfo.split('_')[1] == '1':
        # if there is not 'corrected' in the name, we will store last three elements
        if 'corrected' not in fileinfo:
            info = '_'.join(fileinfo.split('_')[-3:])
        else:
            info = '_'.join(fileinfo.split('_')[-4:])
    elif fileinfo.split('_')[1] == '2':
        # otherwise we store last four elements (5 when corrected)
        if 'corrected' not in fileinfo:
            info = '_'.join(fileinfo.split('_')[-4:])
        else:
            info = '_'.join(fileinfo.split('_')[-5:])

    # Load the balanceboard data
    df_bb = pd.read_csv(bb)

    # Rename columns
    df_bb.columns = ['time_s', 'left_back', 'right_forward', 'right_back', 'left_forward']

    # Calculate sampling rate
    bbsamp = 1 / np.mean(np.diff(df_bb['time_s'] - min(df_bb['time_s'])))

    # Apply Savitzky-Golay filter to smooth the data
    for col in df_bb.columns[1:]:
        df_bb[col] = scipy.signal.savgol_filter(df_bb[col], 51, 5)

    # Calculate COPX and COPY
    COPX = (df_bb['right_forward'] + df_bb['right_back']) - (df_bb['left_forward'] + df_bb['left_back'])
    COPY = (df_bb['right_forward'] + df_bb['left_forward']) - (df_bb['left_back'] + df_bb['right_back'])

    # Calculate COPXc and COPYc
    df_bb['COPXc'] = scipy.signal.savgol_filter(np.insert(np.diff(COPX), 0, 0), 51, 5)
    df_bb['COPYc'] = scipy.signal.savgol_filter(np.insert(np.diff(COPY), 0, 0), 51, 5)

    # Calculate COPc
    df_bb['COPc'] = np.sqrt(df_bb['COPXc']**2 + df_bb['COPYc']**2)

    # restart the time so that starts from 0
    df_bb['time_s'] = df_bb['time_s'] - min(df_bb['time_s'])
    # convert to ms
    df_bb['time_s'] = df_bb['time_s']*1000

    # rename time_s to time
    df_bb.rename(columns={'time_s': 'time'}, inplace=True)

    # Add trialid
    df_bb['TrialID'] = trialid
    # Add info
    df_bb['FileInfo'] = info

    # Write as csv to MTfolder_processed
    df_bb.to_csv(MTfolder_processed + '/bb_' + trialid + '.csv', index=False)

## Plot to check


In [ ]:
import matplotlib.pyplot as plt

bbfiles = glob.glob(MTfolder_processed + '/bb*.csv')

samplebb = pd.read_csv(bbfiles[20])

# plot COPc the sample
plt.plot(samplebb['time'], samplebb['COPc'])
plt.xlim(1000,2000)
plt.show()